In [1]:
%matplotlib inline

In [2]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
import pandas as pd
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.autograd
import torchvision.transforms as T

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class DQN(nn.Module):
    #input dim is 
    def __init__(self, input_dim, hidden_dim, valset_size):
        super(DQN, self).__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.sigmoid = nn.Sigmoid()
        self.hidden2val = nn.Linear(hidden_dim, valset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (Variable(torch.zeros(1, 1, self.hidden_dim)),
                Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, traffic):
        #traffic = torch.unsqueeze(traffic, 1)
        
        lstm_out, self.hidden = self.lstm(traffic, self.hidden)
        #output.append(hx)
        
        val_space = self.hidden2val(lstm_out)
        val_score = val_space[-1]
        return val_score

testdata = Variable(torch.ones((10, 2)))

#print(testdata)   
modeltest = DQN(2, 10, 2)
print(modeltest.forward(testdata))
#print(modeltest.forward(testdata[1]))

Variable containing:
 0.1301  0.1879
[torch.FloatTensor of size 1x2]



In [5]:
#may wanna change end to lower or something 
BATCH_SIZE = 64
GAMMA = 1
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200000


input_dim = 1
hidden_state_dim = 16
valset_dim = 2
model1 = DQN(input_dim, hidden_state_dim, valset_dim)
model2 = DQN(input_dim, hidden_state_dim, valset_dim)
if use_cuda:
    model.cuda()

#think more about what best optimizer to use is
optimizer1 = optim.RMSprop(model1.parameters())
optimizer2 = optim.RMSprop(model2.parameters())
memory = ReplayMemory(10000)


steps_done = 0
            

def valuation(time, USD, ETH, buy_price, sell_price, model):
    context = Variable(state_to_data(time, data))
    model.hidden = model.init_hidden()
    #case where I own ETH
    if (ETH != 0):
        new_USD, _, _ = trade(0, USD, ETH, buy_price, sell_price)
        scale = Variable(torch.FloatTensor([[new_USD], [ETH * sell_price]]))
        #transaction = Variable(torch.FloatTensor([[new_USD - ETH * sell_price], [0]]))
    #case where I own USD
    if (USD != 0):
        _, new_ETH, _ = trade(1, USD, ETH, buy_price, sell_price)
        scale = Variable(torch.FloatTensor([[USD], [new_ETH * sell_price]]))
        #transaction = Variable(torch.FloatTensor([[0], [new_ETH * sell_price - USD]]))
    
    if time == sim_time - 2:
        scale = Variable(torch.FloatTensor([[0], [0]]))
    return scale * model(context).t()
    #return transaction + scale * model(context).t()
    #return scale * model(Variable(context, volatile=True).type(FloatTensor))

def select_action(time, USD, ETH, buy_price, sell_price, model):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        options = valuation(time, USD, ETH, buy_price, sell_price, model)
        if options.data.numpy()[0][0] > options.data.numpy()[1][0]:
            return LongTensor([[0]])
        else:
            return LongTensor([[1]])
    else:
        return LongTensor([[random.randrange(2)]])




In [6]:
last_sync = 0


def optimize_model(i):
    
    use_model1 = (i%10 <= math.ceil(10/2))
    #bool(random.getrandbits(1))
    if use_model1 :
        action_model = model1
        target_model = model2
    else:
        action_model = model2
        target_model = model1
            
    global last_sync
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see http://stackoverflow.com/a/19343/3343043 for
    # detailed explanation).
    batch = Transition(*zip(*transitions))
    
    # Compute a mask of non-final states and concatenate the batch elements
    non_final_mask = ByteTensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)))

    next_statelist = [s for s in batch.next_state if s is not None]
    
    #print(torch.cat([s for s in batch.next_state if s is not None]))
    
    state_batch = batch.state
    action_batch = Variable(torch.cat(batch.action))
    reward_batch = Variable(torch.cat(batch.reward))
    
    #valuation(state, USD, ETH, buy_price, sell_price)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
    
    state_values = Variable(torch.zeros(BATCH_SIZE, 2))
    for i in range(BATCH_SIZE):
        time, USD, ETH = state_batch[i]
        buy_price, sell_price = state_to_prices(time, prices)
        state_values[i] = valuation(time, USD, ETH, buy_price, sell_price, action_model).t()
    
    state_action_values = state_values.gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states.
    next_state_temp = Variable(torch.zeros(BATCH_SIZE, 2), requires_grad = False)
    next_action = ()
    for i in range(BATCH_SIZE):
        time, USD, ETH = next_statelist[i]
        buy_price, sell_price = state_to_prices(time, prices)
        next_state_temp[i] = valuation(time, USD, ETH, buy_price, sell_price, target_model).t()
        next_action = next_action + (select_action(time, USD, ETH, buy_price, sell_price, action_model),)
    
    next_state_values = Variable(torch.zeros(BATCH_SIZE).type(Tensor))
    next_state_values[non_final_mask] = next_state_temp.gather(1, Variable(torch.cat(next_action)))
    
    # Now, we don't want to mess up the loss with a volatile flag, so let's
    # clear it. After this, we'll just end up with a Variable that has
    # requires_grad=False
    
    # next_state_values.volatile = False
    
    # Compute the expected Q values
    
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
  
    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
   
    # Optimize the model
    
    optimizer1.zero_grad()
    
    optimizer2.zero_grad()
    loss.backward()
    
    if use_model1:
        for param in model1.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer1.step()
    else:
        for param in model2.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer2.step()
    
    return loss.data.numpy()
        
    

In [7]:

USD = 100 #amount in USD 
ETH = 0 #amount in ETH 
#0 is action to go USD
#1 is action to go ETH

#all fees use poloniex taker fee. could try a strat that learns live and acts as a maker

buy_pen = 0
#0.25/100 #proportion lost when buying
sell_pen = 0
#0.25/100 #proportion lost when selling

#trade takes in an action, amount and outputs a reward
def trade(action, USD, ETH, buy_price, sell_price):
    if USD > 0:
        currency = 0
    else:
        currency = 1
    if (action == currency):
        return USD, ETH, 0
    elif (action == 1):
        new_ETH = USD * buy_price * (1 - buy_pen)
        new_USD = 0
    elif (action == 0):
        new_USD = ETH * sell_price * (1 - sell_pen)
        new_ETH = 0
    #value of ETH is considered to be whatever you could sell it for to get USD
    return new_USD, new_ETH, 0
    
        

In [8]:
#takes in the state number and gives the context of that trade point
#used in the optimization 
#also needs to give us what currency we own so that can be fed into neurel net as third feature 
def state_to_data(time, data):
    context = data[(time-prep_time):time]
    return context

#takes in state and returns buy and sell price
def state_to_prices(time, prices):
    buy_price = 1 / prices[time]
    sell_price = 1 / buy_price
    return buy_price, sell_price

def state_to_old_prices(time, prices):
    old_buy_price = 1 / prices[time-1]
    old_sell_price = 1 / old_buy_price
    return old_buy_price, old_sell_price
                    



In [9]:
total_data = 720

#df = pd.read_csv('october_array.csv')

#one value was imputed to same price as hour before 
df = pd.read_csv('bitcoin_price_change_running.csv') 


arr = df.values
#data = torch.from_numpy(arr[1:total_data][1:total_data]).float()
data = torch.from_numpy(arr[1:total_data]).float()
print(data)


#df = pd.read_csv('october_data_prices.csv')
df = pd.read_csv('bitcoin_prices.csv')             
arr = df.values
#arr = arr[1:total_data, 2].astype(float)
arr = arr[1:total_data, 0].astype(float)
prices = torch.from_numpy(arr).float()


print(prices)
                 



 4.8412e-03
-3.6294e-04
 1.9215e-02
 8.0952e-03
 8.4539e-03
-4.3918e-03
 4.9594e-03
 5.2247e-03
-4.8047e-03
 8.5035e-04
-1.0789e-04
 4.1111e-04
 7.3303e-03
 5.8133e-03
-9.3264e-04
 5.3416e-03
-3.5509e-04
-6.1865e-03
 3.4028e-02
-3.7299e-02
-1.5621e-02
-1.1544e-02
 7.7399e-03
 1.1395e-02
 8.8990e-03
-5.8890e-03
-5.2300e-03
 9.8365e-03
-4.9783e-03
-5.9446e-03
 7.5999e-03
-2.2587e-03
-7.3184e-04
-1.3272e-02
-8.6080e-03
 1.9967e-03
-1.0968e-02
-1.5590e-03
 7.3148e-03
 1.2585e-04
-7.4267e-04
-1.1100e-03
 1.1013e-03
 2.9188e-03
-4.1744e-03
 1.5176e-03
-5.8913e-03
-8.3874e-04
-3.5729e-03
 1.6055e-02
 1.0728e-02
-3.3085e-03
-1.7772e-04
 1.5052e-03
-1.9116e-03
-6.7387e-03
-3.4208e-03
-2.2926e-03
-2.6114e-03
-5.4553e-03
-2.0984e-02
-1.8702e-02
-5.5386e-03
 2.0136e-03
-9.4588e-04
 1.4881e-02
-1.6368e-02
-2.9776e-02
-3.3562e-03
-1.0391e-02
 1.1103e-02
 1.6642e-02
-8.6015e-03
 3.6530e-03
 2.1278e-02
 5.8693e-03
-3.8821e-03
-2.4107e-03
-5.4906e-03
-2.4205e-03
-6.9070e-03
 6.7415e-03
-5.2991e-04
-6.

In [10]:
import time as timer



def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = timer.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [11]:
#this section runs training. Ignore the profits it spits out because they are influenced by the random decisions.
# Focus on the decreasing loss values

BATCH_SIZE = 500 #setting to smaller number trains faster but diesn't converge as well
start = timer.time()
num_episodes = 300
reg = 5
loss = 0
prep_time = 12 #number of states we don't use at start, to give time for hidden state to init
sim_time = 576 - 2  #total number of time periods
total_profit = [] #stores amount left at end of sim


init_money = 100
for i_episode in range(1, num_episodes + 1):
    
    
    #reset currency amounts
    USD = init_money
    ETH = 0
    #initialize state.
    state = (prep_time, USD, ETH)
    for t in range(prep_time, sim_time - 1):
        #print(state)
        time, USD, ETH = state
        
        #get buy_price and sell_price by indexing into data 
        buy_price, sell_price = state_to_prices(time, prices)
        old_buy_price, old_sell_price = state_to_old_prices(time, prices)
        # Select and perform an action
        action = select_action(time, USD, ETH, buy_price, sell_price, model1)
        #print(action)
        old_USD = USD
        old_ETH = ETH
        
        USD, ETH, _ = trade(action[0,0], USD, ETH, buy_price, sell_price)
        reward = (USD + sell_price*ETH) - (old_USD + old_sell_price*old_ETH)
        
        reward = Tensor([reward])
        
        
        
        #find a way to set done by seeing if we are at the end of the training time (or if 0 money left)
        done = (sim_time - 2 == t) or (USD ==0 and ETH == 0)
        if not done:
            next_state = (time + 1, USD, ETH)
        else:
            next_state = None 

        # Store the transition in memory
        if next_state != None:    
            memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state
        
        if done: 
            # Perform one step of the optimization (on the target network)
            loss += optimize_model(i_episode)
            profit = USD + ETH * sell_price - init_money
            if i_episode%reg == (reg - 1):
                print(profit)
                print('%s (%d %d%%) %.4f' % (timeSince(start, i_episode / num_episodes),
                                         i_episode, i_episode / num_episodes * 100, loss / reg))
                
                loss = 0

            total_profit.append(profit)
            break

print('Complete')
#print(total_profit)

32.930307353837804
0m 46s (- 57m 0s) (4 1%) 18.3578


KeyboardInterrupt: 

In [ ]:
#stuff below is evaluation. Run it to get what the thing would do on the end part of this month's bitcoin data. 

In [31]:

num_episides = 1
prep_time = 12 #number of states we don't use at start, to give time for hidden state to init
train_time = sim_time
sim_time = total_data - 2  #total number of time periods
total_profit = [] #stores amount left at end of sim

#we call our data set history

init_money = 100
for i_episode in range(1):
    
    
    #reset currency amounts
    USD = init_money
    ETH = 0
    #initialize state.
    state = (train_time, USD, ETH)
    for t in range(train_time, sim_time - 1):
        print(state)
        time, USD, ETH = state
    
        
        #get buy_price and sell_price by indexing into data 
        buy_price, sell_price = state_to_prices(time, prices)
        old_buy_price, old_sell_price = state_to_old_prices(time, prices)
        
        #action
        options = valuation(time, USD, ETH, buy_price, sell_price, model1)
        if options.data.numpy()[0][0] > options.data.numpy()[1][0]:
            action = LongTensor([[0]])
        else:
            action = LongTensor([[1]])
            
        #print(action)
        old_USD = USD
        old_ETH = ETH
        
        USD, ETH, _ = trade(action[0,0], USD, ETH, buy_price, sell_price)
        reward = (USD + sell_price*ETH) - (old_USD + old_sell_price*old_ETH)
        
        reward = Tensor([reward])
        
        
        #find a way to set done by seeing if we are at the end of the training time (or if 0 money left)
        done = (sim_time - 2 == t) or (USD ==0 and ETH == 0)
        if not done:
            next_state = (time + 1, USD, ETH)
        else:
            next_state = None 

        # Move to the next state
        state = next_state

        
        
    print(USD + ETH * sell_price - init_money)
    total_profit.append(USD + ETH * sell_price - init_money)
            

print('Complete')
print(total_profit)

(218, 100, 0)
(219, 0, 0.01258534049952126)
(220, 0, 0.01258534049952126)
(221, 0, 0.01258534049952126)
(222, 0, 0.01258534049952126)
(223, 0, 0.01258534049952126)
(224, 0, 0.01258534049952126)
(225, 0, 0.01258534049952126)
(226, 0, 0.01258534049952126)
(227, 0, 0.01258534049952126)
(228, 0, 0.01258534049952126)
(229, 0, 0.01258534049952126)
(230, 0, 0.01258534049952126)
(231, 0, 0.01258534049952126)
(232, 0, 0.01258534049952126)
(233, 0, 0.01258534049952126)
(234, 0, 0.01258534049952126)
(235, 0, 0.01258534049952126)
(236, 0, 0.01258534049952126)
(237, 0, 0.01258534049952126)
(238, 0, 0.01258534049952126)
(239, 0, 0.01258534049952126)
(240, 0, 0.01258534049952126)
(241, 0, 0.01258534049952126)
(242, 0, 0.01258534049952126)
(243, 0, 0.01258534049952126)
(244, 0, 0.01258534049952126)
(245, 0, 0.01258534049952126)
(246, 0, 0.01258534049952126)
(247, 0, 0.01258534049952126)
(248, 0, 0.01258534049952126)
(249, 0, 0.01258534049952126)
(250, 0, 0.01258534049952126)
(251, 0, 0.012585340499521

(539, 0, 0.012564371201547603)
(540, 0, 0.012564371201547603)
(541, 0, 0.012564371201547603)
(542, 0, 0.012564371201547603)
(543, 0, 0.012564371201547603)
(544, 0, 0.012564371201547603)
(545, 0, 0.012564371201547603)
(546, 0, 0.012564371201547603)
(547, 0, 0.012564371201547603)
(548, 0, 0.012564371201547603)
(549, 0, 0.012564371201547603)
(550, 0, 0.012564371201547603)
(551, 0, 0.012564371201547603)
(552, 0, 0.012564371201547603)
(553, 0, 0.012564371201547603)
(554, 0, 0.012564371201547603)
(555, 0, 0.012564371201547603)
(556, 0, 0.012564371201547603)
(557, 0, 0.012564371201547603)
(558, 0, 0.012564371201547603)
(559, 0, 0.012564371201547603)
(560, 0, 0.012564371201547603)
(561, 0, 0.012564371201547603)
(562, 0, 0.012564371201547603)
(563, 0, 0.012564371201547603)
(564, 0, 0.012564371201547603)
(565, 0, 0.012564371201547603)
(566, 0, 0.012564371201547603)
(567, 0, 0.012564371201547603)
(568, 0, 0.012564371201547603)
(569, 0, 0.012564371201547603)
(570, 0, 0.012564371201547603)
(571, 0,

In [29]:
#Look at the reward values for each currency
for i in range(700):
    print(valuation(12+i, 100, 0, 300, 1/300, model1).data.numpy())


[[-18.85990524]
 [-18.61907005]]
[[-18.85963821]
 [-18.58412552]]
[[-18.85898781]
 [-18.58852577]]
[[-18.85890579]
 [-18.62627029]]
[[-18.85928726]
 [-18.59926414]]
[[-18.85926247]
 [-18.62237167]]
[[-18.85977364]
 [-18.65543175]]
[[-18.85933495]
 [-18.45469856]]
[[-18.85804367]
 [-18.79257011]]
[[-18.86209106]
 [-18.73223495]]
[[-18.86340904]
 [-18.68309021]]
[[-18.86246872]
 [-18.57554245]]
[[-18.86015129]
 [-18.54897499]]
[[-18.8585434 ]
 [-18.56945992]]
[[-18.85850716]
 [-18.65327454]]
[[-18.859869  ]
 [-18.65926552]]
[[-18.86022949]
 [-18.57613564]]
[[-18.85952568]
 [-18.63887978]]
[[-18.86013222]
 [-18.65607452]]
[[-18.8603611 ]
 [-18.58639717]]
[[-18.85968018]
 [-18.62646103]]
[[-18.85985756]
 [-18.62771606]]
[[-18.86044502]
 [-18.69051743]]
[[-18.86159897]
 [-18.67247009]]
[[-18.86158752]
 [-18.61104393]]
[[-18.86100006]
 [-18.66992378]]
[[-18.86127663]
 [-18.63326454]]
[[-18.8606205 ]
 [-18.58174515]]
[[-18.85964203]
 [-18.61458206]]
[[-18.85965538]
 [-18.62752533]]
[[-18.8599

[[-18.85958672]
 [-18.60892296]]
[[-18.8595047 ]
 [-18.61688995]]
[[-18.85966301]
 [-18.62905693]]
[[-18.85984802]
 [-18.61392021]]
[[-18.85956001]
 [-18.58715057]]
[[-18.85899544]
 [-18.59042931]]
[[-18.85868073]
 [-18.58403397]]
[[-18.8586483 ]
 [-18.62046051]]
[[-18.85915375]
 [-18.6148243 ]]
[[-18.85930634]
 [-18.60365295]]
[[-18.85943604]
 [-18.64136696]]
[[-18.86001968]
 [-18.63289452]]
[[-18.8602066 ]
 [-18.62686539]]
[[-18.86001968]
 [-18.59931946]]
[[-18.85954475]
 [-18.60723686]]
[[-18.85942841]
 [-18.61413383]]
[[-18.85964012]
 [-18.63778496]]
[[-18.86001015]
 [-18.62022018]]
[[-18.86035156]
 [-18.68056107]]
[[-18.86198425]
 [-18.75813103]]
[[-18.86335373]
 [-18.65450668]]
[[-18.86198235]
 [-18.56217003]]
[[-18.85962105]
 [-18.54204178]]
[[-18.85834885]
 [-18.6045742 ]]
[[-18.85851097]
 [-18.5814476 ]]
[[-18.85858536]
 [-18.62565231]]
[[-18.85923195]
 [-18.6206131 ]]
[[-18.85954857]
 [-18.62128067]]
[[-18.85958672]
 [-18.6016922 ]]
[[-18.8594532 ]
 [-18.62452888]]
[[-18.8595

[[-18.86063004]
 [-18.57969666]]
[[-18.85942078]
 [-18.58178902]]
[[-18.85907555]
 [-18.63524628]]
[[-18.85966301]
 [-18.6234684 ]]
[[-18.85970497]
 [-18.59992599]]
[[-18.85945129]
 [-18.61794853]]
[[-18.85972977]
 [-18.64406967]]
[[-18.86026573]
 [-18.63717079]]
[[-18.86046982]
 [-18.6369381 ]]
[[-18.86034966]
 [-18.60472298]]
[[-18.85986328]
 [-18.61541176]]
[[-18.85951233]
 [-18.57970428]]
[[-18.85892677]
 [-18.60066605]]
[[-18.85909081]
 [-18.6315403 ]]
[[-18.85935974]
 [-18.58006668]]
[[-18.85895729]
 [-18.61462975]]
[[-18.85930252]
 [-18.62965012]]
[[-18.85969162]
 [-18.61746216]]
[[-18.85920525]
 [-18.53359604]]
[[-18.85758018]
 [-18.5200386 ]]
[[-18.85669708]
 [-18.57052994]]
[[-18.85744667]
 [-18.64282227]]
[[-18.858675  ]
 [-18.58472061]]
[[-18.85869789]
 [-18.61657143]]
[[-18.85918617]
 [-18.62812996]]
[[-18.85963821]
 [-18.62308693]]
[[-18.85972977]
 [-18.61010551]]
[[-18.86021423]
 [-18.70909309]]
[[-18.86286354]
 [-18.82732773]]
[[-18.86503601]
 [-18.67769814]]
[[-18.8629